In [1]:
import sys
sys.path.append('/home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *
import math

In [2]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

num_bays = 5        
width = 1.0           
height = 1.0          
length = 1.0          
stiffness = 50000     

cable_len = 2.0       
load_mass = 50.0      

#Calculate diagonal length 
diag_len = math.sqrt(length**2 + height**2)
#Calculate cable diagonal 
cable_diag = math.sqrt((width/2)**2 + cable_len**2)

#fix the corners at X=0
p_bl = mss.add(Fix((0, 0, 0)))          
p_br = mss.add(Fix((0, width, 0)))      
p_tr = mss.add(Fix((0, width, height))) 
p_tl = mss.add(Fix((0, 0, height)))     

# Arm
for i in range(num_bays):
    x = (i + 1) * length
    
    # Standard truss segments
    c_bl = mss.add(Mass(1.0, (x, 0, 0)))         
    c_br = mss.add(Mass(1.0, (x, width, 0)))      
    c_tr = mss.add(Mass(1.0, (x, width, height))) 
    c_tl = mss.add(Mass(1.0, (x, 0, height)))    
    # Length Springs
    mss.add(Spring(length, stiffness, (p_bl, c_bl)))
    mss.add(Spring(length, stiffness, (p_br, c_br)))
    mss.add(Spring(length, stiffness, (p_tr, c_tr)))
    mss.add(Spring(length, stiffness, (p_tl, c_tl)))

    # Square Springs 
    mss.add(Spring(width,  stiffness, (c_bl, c_br))) 
    mss.add(Spring(height, stiffness, (c_br, c_tr))) 
    mss.add(Spring(width,  stiffness, (c_tr, c_tl))) 
    mss.add(Spring(height, stiffness, (c_tl, c_bl))) 

    # Diagonal Springs
    mss.add(Spring(diag_len, stiffness, (p_bl, c_tl))) 
    mss.add(Spring(diag_len, stiffness, (p_br, c_tr))) 
    mss.add(Spring(diag_len, stiffness, (p_tl, c_tr))) 
    mss.add(Spring(diag_len, stiffness, (p_bl, c_br))) 

    # Update 'previous' nodes to be the 'current' ones for the next loop
    p_bl, p_br, p_tr, p_tl = c_bl, c_br, c_tr, c_tl

# Calculate position
tip_x = num_bays * length
load_y = width / 2.0
load_z = height - cable_len

#Heavy Load
load_node = mss.add(Mass(load_mass, (tip_x, load_y, load_z)))

# Connect the Load
mss.add(Spring(cable_diag, stiffness, (p_tl, load_node)))
mss.add(Spring(cable_diag, stiffness, (p_tr, load_node)))

61

In [3]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [4]:
view_width = 800
view_height = 400

camera = PerspectiveCamera( position=[0, -10, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=2.0, position=(0.0, -10.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [5]:
from time import sleep
for i in range(300):
    mss.simulate(0.005, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)